# Web Scraping des offres de Pole-Emploi (et partenaires) par intéractiona avec l'API de Pole-Emploi

In [ ]:
import requests
import pandas as pd
import re

from login_poleemploi import identifiant_poleemploi

In [ ]:
identifiant, password = identifiant_poleemploi() # On cache les identifiants d'accès à pole-emploi pour éviter tout problème de sécurité.
query_params = {"grant_type":"client_credentials",
                "client_id": identifiant, 
                "client_secret": password,
                "scope": "api_offresdemploiv2 o2dsoffre"
}

headers = {"Content-Type": "application/x-www-form-urlencoded"}

Nous devons tout d'abord faire une demande d'un token d'accès pour l'api d'offre d'emploi. 

In [ ]:
# On réalise la requpete pour obtenir une clé d'accès à l'API de Pole-Emploi en utilisant les identifiants obtenus sur le site pole-emploi.io
result = requests.post("https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=%2Fpartenaire", data=query_params, headers=headers)

In [ ]:
result.json()

Nous pouvons ensuite passer à l'extraction des différentes offres.

In [ ]:
# On crée ensuite un dictionnaire pour récupérer les différentes informations
dict_poleemploi = {
    "entreprise": [],
    "titre": [],
    "type_contrat": [],
    "lieu": [],
    "date_publication": [],
    "source": [],
    "link": [],
    "descriptif": [],
    "salaire": [],
    "experience": []
}


In [ ]:
liste_offre = requests.get("https://api.pole-emploi.io/partenaire/offresdemploi/v2/offres/search?motsCles=data&origineOffre=2&sort=1&minCreationDate=2024-01-07T11:10:00Z&maxCreationDate=2024-01-08T06:37:55Z",
                           headers={'Authorization': f'Bearer {result.json()["access_token"]}'})

Les paramètres suivants indiquent donc le mot clé que nous cherchons ("data"), l'origine de l'offre (Pole-Emploi ou partenaires) ainsi que la date minimale et maximal de création de l'offre (attention, l'un oblige la présence de l'autre).

Le nombre d'offres par requête étant limité à 150, nous devons changer manuellement les dates (principalement maximal) pour récupérer toutes les dates petit à petit. Nous réalisons la même manipulation pour les origines de l'offre.

In [ ]:
params = {
    "motsCles": "data",
    "origineOffre": 1,
    "minCreationDate": "2023-01-01T00:01:00Z",
    "maxCreationDate":"2024-01-11T09:00:00Z"
}

In [ ]:
liste_offre = requests.get("https://api.pole-emploi.io/partenaire/offresdemploi/v2/offres/search?",
                           headers={'Authorization': f'Bearer {result.json()["access_token"]}'}, params=params)
liste_offre

In [ ]:
liste_offre.json()

On navie donc ensuite à travers le fichier json pour récupérer les informations qui nous intéressent.

In [ ]:
for item in liste_offre.json()["resultats"]:
    if 'entreprise' in item and 'nom' in item['entreprise']:
        dict_poleemploi["entreprise"].append(item['entreprise']['nom'])
    else: 
        dict_poleemploi["entreprise"].append("Non spécifié")
    dict_poleemploi["titre"].append(item['intitule'])
    if 'typeContrat' in item:
        dict_poleemploi["type_contrat"].append(item['typeContrat'])
    else: 
         dict_poleemploi["type_contrat"].append("Non spécifié")
    if 'lieuTravail' in item and 'codePostal' in item['lieuTravail']:
        dict_poleemploi["lieu"].append(item['lieuTravail']['codePostal'])
    else: 
        dict_poleemploi["lieu"].append(item['lieuTravail']['libelle'])
    if 'partenaires' in item['origineOffre']:
        dict_poleemploi["source"].append(item['origineOffre']['partenaires'][0]["nom"])
    else: 
        dict_poleemploi["source"].append("Pole-Emploi")
    if 'salaire' in item and "libelle" in item["salaire"]:
        dict_poleemploi["salaire"].append(item["salaire"]["libelle"])
    else:
        dict_poleemploi["salaire"].append("Non spécifié")
    if 'experienceLibelle' in item:
        dict_poleemploi["experience"].append(item["experienceLibelle"])
    else:
        dict_poleemploi["experience"].append("Non spécifié") 
    dict_poleemploi["date_publication"].append(item['dateCreation'])
    dict_poleemploi["link"].append(item['origineOffre']['urlOrigine'])
    dict_poleemploi["descriptif"].append(item['description'])
    

In [ ]:
print(len(dict_poleemploi["entreprise"]))

In [ ]:
df_poleemploi = pd.DataFrame(dict_poleemploi)
df_poleemploi.head(10)

In [ ]:
import re

# On crée une petite fonction de nettoyage rapide pour enlever certains patterns de caractères ou espaces en trop.
def clean_string(text):
    cleaned_text = text.replace('\n', ' ')
    cleaned_text = cleaned_text.replace('\t', ' ')
    cleaned_text = cleaned_text.replace('\xa0', ' ')
    cleaned_text = cleaned_text.replace('&nbsp', ' ')
    cleaned_text = cleaned_text.replace('-', ' ')
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    return cleaned_text

In [ ]:
df_poleemploi['descriptif'] = df_poleemploi['descriptif'].apply(clean_string)

In [ ]:
df_poleemploi['descriptif'][1]

Certaines offres divisent bien dans leur texte la partie descriptif de celle du profil requis. Nous divisons donc ce document en deux en utilisant "Profil" comme point de coupure.

In [ ]:
df_poleemploi[['descriptif', 'profil']] = df_poleemploi['descriptif'].str.split('Profil', n=1, expand=True)

In [ ]:
df_poleemploi.head(10)

In [ ]:
df_poleemploi.iloc[1,:]

In [ ]:
# On enregistre le dataframe en un fichier csv en spécifiant bien l'encodage pour garder les caractères spéciaux propre au français.
df_poleemploi.to_csv("poleemploi_11012024.csv", index=None, encoding="utf-8-sig")